In [17]:
import numpy as np
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from timeit import default_timer as timer
from sklearn.preprocessing import normalize

## Q1
Implementing QR method using gram schmidt method for finding Q

In [18]:
def gram_schmidt_algorithm(A):
    n = A.shape[1]
    result = np.zeros_like(A)

    # Orthogonalization
    for i in range(n):
        result[:, i] = A[:, i]
        for j in range(i):
            result[:, i] -= ((A[:, i].T @ result[:, j])[0,0] / (result[:, j].T @ result[:, j])[0,0]) * result[:, j]

    # Normalization
    for i in range(n):
        result[:, i] /= np.linalg.norm(result[:, i])

    return result

def qr_algorithm(A, iterations):
    Q_final = np.eye(A.shape[0])

    # Iterate 'iterations' times and apply QR operation
    for _ in range(iterations):
        Q = gram_schmidt_algorithm(A)
        R = Q.T @ A
        Q_final = Q_final @ Q
        A = R @ Q

    eigenvalues = np.diag(A)
    return eigenvalues, Q_final



In [19]:
A = np.matrix([[2,1,0], [1,3,1], [0,1,4]], dtype=np.float64)
# A = np.matrix([[12,-51,4], [6,167,-68], [-4,24,-41]])
# A = np.matrix([[4,-2,-1],[-2,4,-2],[1,-2,3]], dtype=np.float64)


eig_vals, eig_vecs = qr_algorithm(A, 100)
eig_vals_np = np.linalg.eig(A)[0]


print("QR decomposition  Algorithm: ", eig_vals)
print("function: ", eig_vals_np)
print("Difference: ", np.linalg.norm(eig_vals - eig_vals_np))
print()
print("QR Eigen-vectors : \n", eig_vecs)
print()
print("function Eigen-vectors : \n", np.linalg.eig(A)[1])

QR decomposition  Algorithm:  [4.73205081 3.         1.26794919]
function:  [1.26794919 3.         4.73205081]
Difference:  4.898979485566358

QR Eigen-vectors : 
 [[ 0.21132487 -0.57735027  0.78867513]
 [ 0.57735027 -0.57735027 -0.57735027]
 [ 0.78867513  0.57735027  0.21132487]]

function Eigen-vectors : 
 [[-0.78867513 -0.57735027  0.21132487]
 [ 0.57735027 -0.57735027  0.57735027]
 [-0.21132487  0.57735027  0.78867513]]


We can see that eigen values are approximated nicely. However, the eigenvectors are not the same. it is negative sometimes


## Q2
Implementing Cholesky Factorization

In [20]:

def cholesky_factorization(A):
    if not np.allclose(A, A.T):
        raise ValueError("Matrix is not symmetric")
    if np.any(np.linalg.eigvals(A) <= 0):
        raise ValueError("Matrix is not positive definite")

    n = A.shape[0]
    L = np.zeros_like(A, dtype=float)

    for i in range(n):
        for j in range(i+1):
            if i == j:
                L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :j]**2))
            else:
                L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]
    return L



In [21]:
A = np.array([[4, -1, 1], [-1, 4.25, 2.75], [1, 2.75, 3.5]], dtype=np.float64)
L = cholesky_factorization(A)
print("Original Matrix: \n", A)
print()
print("Cholesky Factorization: \n", L)
print()
print("L @ L.T: \n", L @ L.T)


Original Matrix: 
 [[ 4.   -1.    1.  ]
 [-1.    4.25  2.75]
 [ 1.    2.75  3.5 ]]

Cholesky Factorization: 
 [[ 2.   0.   0. ]
 [-0.5  2.   0. ]
 [ 0.5  1.5  1. ]]

L @ L.T: 
 [[ 4.   -1.    1.  ]
 [-1.    4.25  2.75]
 [ 1.    2.75  3.5 ]]


In [22]:
n = 3
# A = np.random.rand(n, n,)
L = cholesky_factorization(A)

## Q3
Solving follwing equations

6x + 15y + 55z = 76

15x + 55y + 225z = 295

55x + 225y + 979z = 1259

In [23]:

def forward_substitution(L, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    return y

def backward_substitution(U, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x


In [24]:
A = np.array([[6,15,55],
              [15,55,225],
              [55,225,979]])
B = np.array([[76],[295],[1259]])


solving Ax = b in 2 steps
- L @ y = B
- L.T @ x = y 

In [25]:
L = cholesky_factorization(A)
y = forward_substitution(L,B)
x = backward_substitution(L.T,y)
print("Soltion to the system: \n", x)

Soltion to the system: 
 [1. 1. 1.]


In [30]:
print(L)

[[ 2.44948974  0.          0.        ]
 [ 6.12372436  4.18330013  0.        ]
 [22.45365598 20.91650066  6.11010093]]


## Q4


#### a

In [26]:
# Load the Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)

#### b

In [31]:
# normalize each feature vector using L1 norm
X_train_l1 = X_train / np.linalg.norm(X_train, ord=1, axis=0)
X_test_l1 = X_test / np.linalg.norm(X_test, ord=1, axis=0)

# normalize each feature vector using L1 norm
X_train_l2 = X_train / np.linalg.norm(X_train, ord=2, axis=0)
X_test_l2 = X_test / np.linalg.norm(X_test, ord=2, axis=0)


[6.40232400e+03 8.79908000e+03 4.16757300e+04 2.95647600e+05
 4.37451000e+01 4.72683000e+01 4.04318867e+01 2.21191920e+01
 8.21108000e+01 2.85900400e+01 1.79431900e+02 5.56506000e+02
 1.27098500e+03 1.76176330e+04 3.16386400e+00 1.16077000e+01
 1.47046627e+01 5.33798700e+00 9.26318200e+00 1.73330950e+00
 7.35790000e+03 1.17117500e+04 4.85092000e+04 3.96219200e+05
 6.01149200e+01 1.15674190e+02 1.24432971e+02 5.18655810e+01
 1.31329600e+02 3.81736600e+01]


#### c

In [28]:
# original Data
tim = timer()
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
tim_end = timer()
print("Time taken for original data: ", tim_end - tim)


# L1 normalized Data
tim = timer()
clf_l1 = SVC(kernel='linear')
clf_l1.fit(X_train_l1, y_train)
y_pred_l1 = clf_l1.predict(X_test_l1)
tim_end = timer()
print("Time taken for L1 Normalized data: ", tim_end - tim)

# L2 normalized Data
tim = timer()
clf_l2 = SVC(kernel='linear')
clf_l2.fit(X_train_l2, y_train)
y_pred_l2 = clf_l2.predict(X_test_l2)
tim_end = timer()
print("Time taken for L2 Normalized data: ", tim_end - tim)



Time taken for original data:  1.8378897999999992
Time taken for L1 Normalized data:  0.014183900000006133
Time taken for L2 Normalized data:  0.00997389999997722


In [29]:
# Acuracy
print("Accuracy score:")
print("\tOriginal Data: ", accuracy_score(y_test, y_pred))
print("\tL1 Normalized Data: ", accuracy_score(y_test, y_pred_l1))
print("\tL2 Normalized Data: ", accuracy_score(y_test, y_pred_l2))
print()

# Precision
print("Precision score:")
print("\tOriginal Data: ", precision_score(y_test, y_pred,))
print("\tL1 Normalized Data: ", precision_score(y_test, y_pred_l1,))
print("\tL2 Normalized Data: ", precision_score(y_test, y_pred_l2,))
print()

# Recall
print("Recall score:")
print("\tOriginal Data: ", recall_score(y_test, y_pred,))
print("\tL1 Normalized Data: ", recall_score(y_test, y_pred_l1,))
print("\tL2 Normalized Data: ", recall_score(y_test, y_pred_l2,))



Accuracy score:
	Original Data:  0.9736842105263158
	L1 Normalized Data:  0.5789473684210527
	L2 Normalized Data:  0.6929824561403509

Precision score:
	Original Data:  0.9701492537313433
	L1 Normalized Data:  0.5789473684210527
	L2 Normalized Data:  1.0

Recall score:
	Original Data:  0.9848484848484849
	L1 Normalized Data:  1.0
	L2 Normalized Data:  0.4696969696969697


Training Duration
Once the data is normalized, the training time was notably reduced, as mentioned earlier. It's almost a 1:100 ratio between normalized and non-normalized data.

Accuracy Evaluation
We can observe that post normalization, the accuracy score of the model significantly decreased, which is unexpected.

Precision Evaluation
The precision score of the model decreased after normalization for L1 regularization, but for L2 regularization, it increased to a perfect 1.

Recall Evaluation
The recall score of the model decreased after normalization for L2 regularization, but for L1 regularization, it increased to a perfect 1.
